### You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/trippy_extended_tutorial.ipynb).

# TripPy Goal oriented bot in DeepPavlov

This tutorial describes how to build an **advanced** Goal-Oriented Bot (Gobot) in DeepPavlov using the [TripPy architecture](https://arxiv.org/pdf/2005.02877.pdf).
You can also train a simple bot following the trippy_simple tutorial.


This tutorial follows the same structure & uses the same data as the gobot_extended tutorial. We will only go over TripPy specific points here - so consult the gobot_extended notebook for general insights. Note that the only difference is the config used and fewer steps being needed for TripPy.

0. [Data preparation](#scrollTo=4R066YWhTgU6)
1. [Build Database of items](#scrollTo=l5mjRphbTgVb)
2. [Build and Train a Bot](#scrollTo=E_InRKO6TgWt)
3. [Interact with bot](#scrollTo=ElGD1tnJTgYC)
4. [Integrate Google Maps API into Bot](#scrollTo=DH1KPe1fQMfE)
5. [Interact with bot on Telegram](#scrollTo=YdMfO4o0QnK2)

In [1]:
!git clone -b rulebased_gobot_trippy https://github.com/Muennighoff/DeepPavlov
%cd DeepPavlov
!pip install -r requirements.txt
!pip install transformers==2.9.1

Cloning into 'DeepPavlov'...
remote: Enumerating objects: 58560, done.
remote: Counting objects: 100% (1503/1503), done.
remote: Compressing objects: 100% (494/494), done.
remote: Total 58560 (delta 1142), reused 1295 (delta 996), pack-reused 57057
Receiving objects: 100% (58560/58560), 37.82 MiB | 23.73 MiB/s, done.
Resolving deltas: 100% (44987/44987), done.
/content/DeepPavlov
     |████████████████████████████████| 40 kB 23 kB/s 
     |████████████████████████████████| 2.1 MB 12.1 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 2.9 MB 45.2 MB/s 
     |████████████████████████████████| 1.5 MB 45.9 MB/s 
     |████████████████████████████████| 20.1 MB 58.9 MB/s 
     |████████████████████████████████| 10.4 MB 37.1 MB/s 
     |████████████████████████████████| 510 kB 43.3 MB/s 
     |████████████████████████████████| 7.3 MB 22.9 MB/s 
     |████████████████████████████████| 46 kB 4.3 MB/s 
     |████████████████████████████████| 8.

     |████████████████████████████████| 641 kB 8.0 MB/s 
     |████████████████████████████████| 5.6 MB 21.9 MB/s 
     |████████████████████████████████| 1.2 MB 47.5 MB/s 


## 0. Data Preparation

In [2]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader

data = SimpleDSTC2DatasetReader().read('my_data')

2021-08-04 17:56:20.666 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 304: [loading dialogs from my_data/simple-dstc2-trn.json]
2021-08-04 17:56:20.676 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 304: [loading dialogs from my_data/simple-dstc2-val.json]
2021-08-04 17:56:20.738 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 304: [loading dialogs from my_data/simple-dstc2-tst.json]
2021-08-04 17:56:20.843 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 296: There are 479 samples in train split.
2021-08-04 17:56:20.844 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 297: There are 6231 samples in valid split.
2021-08-04 17:56:20.846 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 298: There are 6345 samples in test split.


In [3]:
!ls my_data

dstc2-templates.txt  resto.sqlite		 simple-dstc2-tst.json
dstc2-trn.jsonlist   simple-dstc2-templates.txt  simple-dstc2-val.json
dstc2-tst.jsonlist   simple-dstc2-trn.full.json
dstc2-val.jsonlist   simple-dstc2-trn.json


To iterate over batches of preprocessed DSTC-2 we need to import `DatasetIterator`.

In [4]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

You can now iterate over batches of preprocessed DSTC-2 dialogs:

In [5]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

User utterances:
----------------

[   {'prev_resp_act': None, 'text': ''},
    {   'prev_resp_act': 'welcomemsg',
        'slots': [['food', 'australian'], ['pricerange', 'expensive']],
        'text': 'i am looking for an expensive restaurant that serves '
                'australian food'},
    {   'prev_resp_act': 'confirm-domain',
        'slots': [['food', 'australian'], ['pricerange', 'expensive']],
        'text': 'i am looking for an expensive restaurant that serves '
                'australian food'},
    {'prev_resp_act': 'expl-conf_pricerange', 'text': 'ye'},
    {'db_result': {}, 'prev_resp_act': 'api_call', 'text': 'ye'},
    {   'prev_resp_act': 'canthelp_food_pricerange',
        'slots': [['food', 'asian oriental']],
        'text': 'how about asian oriental food'},
    {   'db_result': {   'addr': '169 high street chesterton chesterton',
                         'area': 'north',
                         'food': 'asian oriental',
                         'name': 'saig

In real-life annotation of data is expensive. To make our tutorial closer to production use-cases we take  only 50 dialogues for training.

In [6]:
!cp my_data/simple-dstc2-trn.json my_data/simple-dstc2-trn.full.json

In [7]:
import json

NUM_TRAIN = 50

with open('my_data/simple-dstc2-trn.full.json', 'rt') as fin:
    data = json.load(fin)
with open('my_data/simple-dstc2-trn.json', 'wt') as fout:
    json.dump(data[:NUM_TRAIN], fout, indent=2)
print(f"Train set is reduced to {NUM_TRAIN} dialogues (out of {len(data)}).")

Train set is reduced to 50 dialogues (out of 50).


## 1. Build Database of items

### Building database of restaurants

In [8]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(primary_keys=["name"],
                           save_path="my_bot/db.sqlite")

2021-08-04 17:56:21.957 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2021-08-04 17:56:21.961 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /content/DeepPavlov/my_bot/db.sqlite.


In [9]:
db_results = []

for dialog in iterator.gen_batches(batch_size=1, data_type='all'):
    turns_x, turns_y = dialog
    db_results.extend(x['db_result'] for x in turns_x[0] if x.get('db_result'))

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results)

Adding 1780 items.


### Interacting with database

We can now play with the database and make requests to it:

In [10]:
database([{'pricerange': 'cheap', 'area': 'south'}])

[[{'addr': 'cambridge leisure park clifton way',
   'area': 'south',
   'food': 'portuguese',
   'name': 'nandos',
   'phone': '01223 327908',
   'postcode': 'c.b 1, 7 d.y',
   'pricerange': 'cheap'},
  {'addr': 'cambridge leisure park clifton way cherry hinton',
   'area': 'south',
   'food': 'chinese',
   'name': 'the lucky star',
   'phone': '01223 244277',
   'postcode': 'c.b 1, 7 d.y',
   'pricerange': 'cheap'}]]

## 3. Build and Train a Bot

The below image comes from the [TripPy paper](https://arxiv.org/pdf/2005.02877.pdf) and sketches out the models architecture.

&nbsp;
![trippy_architecture_original.png](https://github.com/Muennighoff/DeepPavlov/blob/rulebased_gobot_trippy/examples/img/trippy_architecture_original.jpg?raw=1)
&nbsp;

The entire dialogue history, the last system & user utterances are tokenized and fed into a [BERT Model](https://arxiv.org/pdf/1810.04805.pdf). The model makes use of attention to calculate the importance of tokens in the input. In TripPy the BERT model is trained to do binary clasification for each input token in regards to whether it is a slot value of one of the predefined slot names.

For example, for the slot name "pricerange" the model will look at each token and classify whether it corresponds to that slot. For the input: *I want cheap food*, the output for pricerange should be [0,0,1,0], hence identifying that cheap corresponds to the pricerange. This span prediction is then used to copy the value out of the input.

Apart from "span" (also called "copy_value"), other "class types" (Predictions made for each slot name) are: 
- "dontcare" The model thinks the user does not care about this slot name's value
- "none": The user has not yet indicated his preference for this slot name
- "refer": The user has indicated his preference via another slot name
- "inform": The model has previously informed the user about the slot name
- "true / false": Used when there are slotnames with boolean values

Below is a sketch for how the full TripPy model has been implemented in DeepPavlov:

&nbsp;
![trippy_architecture.png](https://github.com/Muennighoff/DeepPavlov/blob/rulebased_gobot_trippy/examples/img/trippy_architecture.png?raw=1)
&nbsp;

The above image also includes the input & input processing steps, while the previous sketch starts with the BERT Model (BERTForDST). 
Novel things in the DeepPavlov TripPy implementation are:
- The preprocessing is robust to datasets which do not contain position labels (During training TripPy requires position labels to train up its copy value capabilities) - This has been done by calculating Levenshtein distances
- An action prediction head has been added, which predicts what action the system should take from a predefined list of actions
- A database connection has been added, which allows the model to retrieve information about slot values from an sqlite Database
- A Natural Language Generation component has been added, which takes in the predicted action and database results and puts together the final response tothe user


We will now proceed with configuring the model & training.

In [11]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

# Use TripPy Config
gobot_config = read_json(configs.go_bot.trippy_dstc2_minimal)

gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_type'] = 'DefaultTemplate'
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_path'] = 'my_data/simple-dstc2-templates.txt'

gobot_config['metadata']['variables']['DATA_PATH'] = 'my_data'
gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'



Configure bot to use our database:

In [12]:
gobot_config['chainer']['pipe'][-1]['database'] = {
    'class_name': 'sqlite_database',
    'primary_keys': ["name"],
    'save_path': 'my_bot/db.sqlite'
}

Configure bot to use templates:

In [13]:
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_type'] = 'DefaultTemplate'
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_path'] = 'my_data/simple-dstc2-templates.txt'

Specify train/valid/test data path and path to save the final bot model:

In [14]:
gobot_config['metadata']['variables']['DATA_PATH'] = 'my_data'
gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'
# Configure the possible slot names - The "this" slotname is meaningless, but it is somehow part of the training set
gobot_config['chainer']['pipe'][-1]['slot_names'] = ['pricerange', 'this', 'area', 'food']

In [ ]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 4 # set batch size - Ideally use 8 & set lr to 1e-4 if your GPU allows
gobot_config['train']['max_batches'] = 600 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 40 # evaluate on full 'valid' split every x epochs
gobot_config['train']['log_every_n_batches'] = 40 # evaluate on full 'train' split every x batches
gobot_config['train']['validation_patience'] = 10 # evaluate on full 'valid' split every x epochs
gobot_config['train']['log_on_k_batches'] = 10 # How many batches to use for logging

gobot_config['chainer']['pipe'][-1]['debug'] = False
gobot_config['chainer']['pipe'][-1]["optimizer_parameters"] = {"lr": 1e-5, "eps": 1e-6}

train_model(gobot_config)

Optionally, you can download the pre-trained model from kaggle. You will need a kaggle account and to upload your kaggle.json file. Then you may have to run the below cell two times.

In [18]:
### Optional - Download Pretrained TripPy from kaggle ###

# Make your json accessible to kaggle
#!cp /content/kaggle.json /root/.kaggle/

# Download the dataset
#!kaggle datasets download -d muennighoff/trippy-restaurant
#!unzip trippy-restaurant.zip

# Move into correct directory
#!mv db.sqlite /content/DeepPavlov/my_bot/
#!mv model.pth.tar /content/DeepPavlov/my_bot/

trippy-restaurant.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  trippy-restaurant.zip
  inflating: db.sqlite               
  inflating: model.pth.tar           


### Evaluation of training

Calculating **accuracy** of trained bot: whether predicted system responses match true responses (full string match).

In [ ]:
from deeppavlov import evaluate_model

evaluate_model(gobot_config);

With settings of `max_batches=800`, valid accuracy `= 0.44` and test accuracy is `~ 0.45`.


If you have the compute, try training the model with a higher batch size, such as 8, or 16. The code automatically detects multiple GPUs and will run Data Parallelism. You will, however, need to upgrade the transformers huggingface version to 4.X and fix two transformer import statements in the TripPy code that will be raised.



## 4. Interact with Bot

In [19]:
from deeppavlov import build_model

bot = build_model(gobot_config)

2021-08-04 18:17:39.8 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2021-08-04 18:17:39.11 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /content/DeepPavlov/my_bot/db.sqlite.
2021-08-04 18:17:43.393 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 159: Load path /content/DeepPavlov/my_bot/model is given.
2021-08-04 18:17:43.395 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 166: Load path /content/DeepPavlov/my_bot/model.pth.tar exists.
2021-08-04 18:17:43.402 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 167: Initializing `TripPy` from saved.
2021-08-04 18:17:43.403 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 170: Loading weights from /content/DeepPavlov/my_bot/model.pth.tar.
2021-08-04 18:17:44.545 INFO in 'deeppavlov.core.models.torch_model'['torch_model'] at line 98

In [20]:
bot.reset()
bot(['hi, i want to eat, can you suggest a place to go?'])

[['What kind of food would you like?']]

In [21]:
bot(['Perhaps something cheap'])

[['What part of town do you have in mind?']]

In [22]:
bot(['In the north of town'])

2021-08-04 18:17:53.170 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 397: Made api_call with dict_keys(['pricerange', 'this', 'area', 'food']), got 11 results.


[['api_call area="north" food="dontcare" pricerange="dontcare"',
  'Meghna is a nice place in the north of town and the prices are moderate.']]

In [23]:
bot(['Whats their phone number?'])

[['The phone number of meghna is 01223 727410.']]

In [24]:
bot(['and the address?'])

[['Sure, meghna is on 205 victoria road chesterton.']]

In [25]:
bot(['whats their pricerange again?'])

[['The price range at meghna is moderate.']]

In [26]:
bot(['Alright sounds good, thank you!'])

[['You are welcome!']]

## 5. Integrate Google Maps API into Bot

In this part we show how you can add any external API to your Go-bot. To make use of the same training data, we could query Google Maps for restaurants instead of the Database we set up.

First we define the **make_api_call** function, which is responsible for implementing the logic to call the API. The **fill_current_state_with_db_results** is responsible for filling the models Dialogue State with the results from the make_api_call function.

Via self.XXX, both functions can access all attributes of TripPy.

In [31]:
import requests

def make_api_call(self) -> None:

    # Definitions
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

    # Pase your Google Maps API Key with access to the Places API here:
    api_key = "YOUR_API_KEY"
    # Alternatively paste your key in a .txt file and just read it
    with open('/content/gapikey.txt', 'r') as f:
        api_key = f.read()

    # Google Maps returns a number, but our dataset works with textual descriptions of price
    price_map = {
        "cheap": ("0","2"),
        "moderate": ("2","4"),
        "expensive": ("3","5"),
        "dontcare": ("0","5")
    }

    # Format the URL query
    # Make query e.g. "japanese food"
    query = self.ds.get("food", "any") + " food"
    minprice, maxprice = price_map.get(self.ds.get("pricerange", "dontcare"))

    # Send & get results
    r = requests.get(url + 'query=' + query +
                        '&minprice=' + minprice +
                        '&maxprice=' + maxprice +
                        '&type=restaurant' +
                        '&key=' + api_key)
    
    result = r.json()["results"][0]
    # Add food to the result, as not returned by Google
    result["food"] = query.split(" ")[0]
    self.db_result = result

def fill_current_state_with_db_results(self) -> None:

    if self.db_result:
        inv_price_map = {
        "0": "cheap",
        "1": "cheap",
        "2": "moderate",
        "3": "moderate",
        "4": "expensive",
        "5": "expensive",
        }

        # In training data:
        #'db_result': {   'addr': 'regent street city centre',
        #                     'area': 'centre',
        #                     'food': 'italian',
        #                     'name': 'pizza hut city centre',
        #                     'phone': '01223 323737',
        #                     'pricerange': 'cheap'},

        self.ds["addr"] = self.db_result["formatted_address"]
        self.ds["food"] = self.db_result["food"]
        self.ds["name"] = self.db_result["name"]
        self.ds["pricerange"] = inv_price_map.get(self.db_result["price_level"], "moderate")

        # Additionally you could geo-decode the geometry paramter to get an area
        # & to get the phone number we'd have to do another API req, see
        # https://stackoverflow.com/questions/46752928/how-to-get-the-phonenumber-using-google-places-api

In [32]:
# Add the functions to the config
gobot_config['chainer']['pipe'][-1]['make_api_call'] = make_api_call
gobot_config['chainer']['pipe'][-1]['fill_current_state_with_db_results'] = fill_current_state_with_db_results

In [33]:
from deeppavlov import build_model
bot = build_model(gobot_config)

2021-08-04 18:20:04.380 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2021-08-04 18:20:04.383 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /content/DeepPavlov/my_bot/db.sqlite.
2021-08-04 18:20:08.691 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 159: Load path /content/DeepPavlov/my_bot/model is given.
2021-08-04 18:20:08.693 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 166: Load path /content/DeepPavlov/my_bot/model.pth.tar exists.
2021-08-04 18:20:08.696 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 167: Initializing `TripPy` from saved.
2021-08-04 18:20:08.699 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 170: Loading weights from /content/DeepPavlov/my_bot/model.pth.tar.
2021-08-04 18:20:09.462 INFO in 'deeppavlov.core.models.torch_model'['torch_model'] at line

In [29]:
bot.reset()
bot(['hi, i want to eat, can you suggest a place to go?'])

[['What kind of food would you like?']]

In [30]:
bot(['Hmmmm i want chinese food'])

[['api_call area="dontcare" food="chinese" pricerange="dontcare"',
  'Great China Restaurant serves chinese food.']]

In [31]:
bot(['Whats their address?'])
# Try look up the address online! You'll find it on GMaps =)

[['Sure, Great China Restaurant is on 2001 W Main St, Independence, KS 67301, United States.']]

In [32]:
bot(['And their pricerange?'])

[['The price range at Great China Restaurant is moderate.']]

In [33]:
bot(['What food do they serve again?'])

[['Great China Restaurant serves chinese food.']]

In [34]:
bot(['Okay, thanks bye!'])

[['You are welcome!']]

## 6. Interact with Bot on Telegram

In [39]:
bot.reset()

In [40]:
from deeppavlov.utils.telegram import interact_model_by_telegram

telegram_token = "YOUR_TELEGRAM_TOKEN_HERE"

interact_model_by_telegram(model_config=gobot_config, token=telegram_token)

2021-08-04 18:21:57.276 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2021-08-04 18:21:57.282 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /content/DeepPavlov/my_bot/db.sqlite.
2021-08-04 18:22:01.282 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 159: Load path /content/DeepPavlov/my_bot/model is given.
2021-08-04 18:22:01.284 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 166: Load path /content/DeepPavlov/my_bot/model.pth.tar exists.
2021-08-04 18:22:01.289 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 167: Initializing `TripPy` from saved.
2021-08-04 18:22:01.293 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 170: Loading weights from /content/DeepPavlov/my_bot/model.pth.tar.
2021-08-04 18:22:02.126 INFO in 'deeppavlov.core.models.torch_model'['torch_model'] at line

See here for an example image of chatting with the bot in Telegram:

&nbsp;
![trippy_telegram.jpg](https://github.com/Muennighoff/DeepPavlov/blob/rulebased_gobot_trippy/examples/img/trippy_telegram.jpg?raw=1)
&nbsp;


There's still a lot to improve, such as
- Allow the user to share his location
- Format the bot's outputs
- Perhaps even connect with a second API such as AirTable for booking

... Have fun exploring & feel free to submit a PR to further enhance this Notebook =)